In [47]:
import pandas as pd

df_peakonly = pd.read_csv("D:/ms-peakonly-main/peakonly_results/peakonly_QE/QESA1.csv")
df_feature = pd.read_csv("D:/QuanFormer-main/data/FeatureQE.csv")

In [48]:
df_peakonly

,peak_label,mz_mean,mz_width,rt,rt_min,rt_max,D:/QE/SA1.mzML
0,100.014@0.48,100.014487,10,0.476380,0.441858,0.510901,2.524339e+07
1,100.014@0.57,100.014487,10,0.570939,0.528913,0.612965,4.369665e+07
2,100.016@28.42,100.015577,10,28.418450,28.358740,28.478159,2.878832e+06
3,100.016@0.48,100.016177,10,0.480882,0.441858,0.519907,1.521488e+07
4,100.016@0.58,100.016177,10,0.579945,0.561933,0.597956,8.311061e+06
...,...,...,...,...,...,...,...
41430,1492.55@30.32,1492.549550,10,30.320815,30.279834,30.361796,4.043300e+05
41431,1492.565@30.32,1492.565024,10,30.320815,30.279834,30.361796,5.507371e+05
41432,1492.58@30.32,1492.580498,10,30.320815,30.279834,30.361796,4.607344e+05
41433,1492.596@30.32,1492.595972,10,30.320815,30.279834,30.361796,2.757230e+05


In [49]:
df_feature

,mz,RT,Compound Name,formula,Fold change,p-value,Compound concentration ratio
0,371.1617,18.85,1,C21H26N2S2,0.47,0.000,1/2
1,442.2233,17.66,2,C25H31NO6,21.23,0.000,16/1
2,292.1118,3.01,3,C14H17N3O2S,1.91,0.019,2/1
3,502.2395,22.78,4,C27H31N7OS,1.06,0.290,1/1
4,342.1706,3.37,5,C20H23NO4,1.07,0.399,1/1
...,...,...,...,...,...,...,...
831,392.2339,7.05,832,C24H29N3O2,1.00,0.931,1/1
832,310.1254,7.73,833,C19H19NOS,1.06,0.204,1/1
833,242.1175,25.26,834,C15H15NO2,0.99,0.853,1/1
834,242.1179,26.45,835,C15H15NO2,1.05,0.561,1/1


In [50]:
df_feature["RT_min"] = df_feature["RT"]-0.5
df_feature["RT_max"] = df_feature["RT"]+0.5

In [51]:
def calculate_mz_range(mz, ppm=25):
    mz_min = mz * (1 - ppm / 1e6)
    mz_max = mz * (1 + ppm / 1e6)
    return mz_min, mz_max
df_feature['mz_min'], df_feature['mz_max'] = zip(*df_feature['mz'].apply(calculate_mz_range))

In [52]:
df_feature

,mz,RT,Compound Name,formula,Fold change,p-value,Compound concentration ratio,RT_min,RT_max,mz_min,mz_max
0,371.1617,18.85,1,C21H26N2S2,0.47,0.000,1/2,18.35,19.35,371.152421,371.170979
1,442.2233,17.66,2,C25H31NO6,21.23,0.000,16/1,17.16,18.16,442.212244,442.234356
2,292.1118,3.01,3,C14H17N3O2S,1.91,0.019,2/1,2.51,3.51,292.104497,292.119103
3,502.2395,22.78,4,C27H31N7OS,1.06,0.290,1/1,22.28,23.28,502.226944,502.252056
4,342.1706,3.37,5,C20H23NO4,1.07,0.399,1/1,2.87,3.87,342.162046,342.179154
...,...,...,...,...,...,...,...,...,...,...,...
831,392.2339,7.05,832,C24H29N3O2,1.00,0.931,1/1,6.55,7.55,392.224094,392.243706
832,310.1254,7.73,833,C19H19NOS,1.06,0.204,1/1,7.23,8.23,310.117647,310.133153
833,242.1175,25.26,834,C15H15NO2,0.99,0.853,1/1,24.76,25.76,242.111447,242.123553
834,242.1179,26.45,835,C15H15NO2,1.05,0.561,1/1,25.95,26.95,242.111847,242.123953


In [53]:
filtered_df = pd.DataFrame()
for index, feature_row in df_feature.iterrows():
    mz_min, mz_max = feature_row['mz_min'], feature_row['mz_max']
    rt_min, rt_max = feature_row['RT_min'], feature_row['RT_max']
    temp_df = df_peakonly[
        (df_peakonly['mz_mean'] >= mz_min) &
        (df_peakonly['mz_mean'] <= mz_max) &
        (df_peakonly['rt'] >= rt_min) &
        (df_peakonly['rt'] <= rt_max)
    ]
    
    if not temp_df.empty:
        temp_df['feature_index'] = index
        filtered_df = pd.concat([filtered_df, temp_df])

merged_df = pd.merge(filtered_df, df_feature, left_on='feature_index', right_index=True, how='left')

C:\Users\zhangzhengyi\AppData\Local\Temp\ipykernel_22860\3890527394.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['feature_index'] = index
C:\Users\zhangzhengyi\AppData\Local\Temp\ipykernel_22860\3890527394.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['feature_index'] = index
C:\Users\zhangzhengyi\AppData\Local\Temp\ipykernel_22860\3890527394.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

C:\Users\zhangzhengyi\AppData\Local\Temp\ipykernel_22860\3890527394.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['feature_index'] = index
C:\Users\zhangzhengyi\AppData\Local\Temp\ipykernel_22860\3890527394.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['feature_index'] = index
C:\Users\zhangzhengyi\AppData\Local\Temp\ipykernel_22860\3890527394.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

C:\Users\zhangzhengyi\AppData\Local\Temp\ipykernel_22860\3890527394.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['feature_index'] = index
C:\Users\zhangzhengyi\AppData\Local\Temp\ipykernel_22860\3890527394.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['feature_index'] = index
C:\Users\zhangzhengyi\AppData\Local\Temp\ipykernel_22860\3890527394.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

C:\Users\zhangzhengyi\AppData\Local\Temp\ipykernel_22860\3890527394.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['feature_index'] = index
C:\Users\zhangzhengyi\AppData\Local\Temp\ipykernel_22860\3890527394.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['feature_index'] = index
C:\Users\zhangzhengyi\AppData\Local\Temp\ipykernel_22860\3890527394.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

C:\Users\zhangzhengyi\AppData\Local\Temp\ipykernel_22860\3890527394.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['feature_index'] = index
C:\Users\zhangzhengyi\AppData\Local\Temp\ipykernel_22860\3890527394.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['feature_index'] = index
C:\Users\zhangzhengyi\AppData\Local\Temp\ipykernel_22860\3890527394.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [54]:
merged_df

,peak_label,mz_mean,mz_width,rt,rt_min,rt_max,D:/QE/SA1.mzML,feature_index,mz,RT,Compound Name,formula,Fold change,p-value,Compound concentration ratio,RT_min,RT_max,mz_min,mz_max
12096,371.154@19.07,371.154239,10,19.067405,18.845353,19.289457,2.571572e+08,0,371.1617,18.85,1,C21H26N2S2,0.47,0.000,1/2,18.35,19.35,371.152421,371.170979
12103,371.161@19.09,371.160970,10,19.089116,18.869354,19.308879,6.198488e+09,0,371.1617,18.85,1,C21H26N2S2,0.47,0.000,1/2,18.35,19.35,371.152421,371.170979
12107,371.167@19.07,371.166562,10,19.067315,18.831724,19.302907,9.628062e+08,0,371.1617,18.85,1,C21H26N2S2,0.47,0.000,1/2,18.35,19.35,371.152421,371.170979
18031,442.215@17.66,442.214754,10,17.661341,17.496447,17.826236,2.227147e+07,1,442.2233,17.66,2,C25H31NO6,21.23,0.000,16/1,17.16,18.16,442.212244,442.234356
18034,442.222@17.64,442.222117,10,17.642264,17.433237,17.851292,3.657585e+08,1,442.2233,17.66,2,C25H31NO6,21.23,0.000,16/1,17.16,18.16,442.212244,442.234356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4247,242.118@25.27,242.117630,10,25.273209,25.190713,25.355705,1.231051e+07,833,242.1175,25.26,834,C15H15NO2,0.99,0.853,1/1,24.76,25.76,242.111447,242.123553
4249,242.118@26.46,242.117798,10,26.460919,26.352150,26.569688,1.912972e+07,834,242.1179,26.45,835,C15H15NO2,1.05,0.561,1/1,25.95,26.95,242.111847,242.123953
22029,484.116@11.97,484.116272,10,11.970737,11.864650,12.076825,8.107271e+06,835,484.1236,11.84,836,C24H26BrN3O3,1.01,0.899,1/1,11.34,12.34,484.111497,484.135703
22034,484.124@11.97,484.124065,10,11.969091,11.840145,12.098037,4.318005e+07,835,484.1236,11.84,836,C24H26BrN3O3,1.01,0.899,1/1,11.34,12.34,484.111497,484.135703


In [55]:
columns_to_drop = ['peak_label','mz_mean','rt','rt_min','rt_max','mz_width','RT_min', 'RT_max', 'mz_min', 'mz_max', 'feature_index']
merged_df.drop(columns=columns_to_drop, inplace=True)
merged_df

,D:/QE/SA1.mzML,mz,RT,Compound Name,formula,Fold change,p-value,Compound concentration ratio
12096,2.571572e+08,371.1617,18.85,1,C21H26N2S2,0.47,0.000,1/2
12103,6.198488e+09,371.1617,18.85,1,C21H26N2S2,0.47,0.000,1/2
12107,9.628062e+08,371.1617,18.85,1,C21H26N2S2,0.47,0.000,1/2
18031,2.227147e+07,442.2233,17.66,2,C25H31NO6,21.23,0.000,16/1
18034,3.657585e+08,442.2233,17.66,2,C25H31NO6,21.23,0.000,16/1
...,...,...,...,...,...,...,...,...
4247,1.231051e+07,242.1175,25.26,834,C15H15NO2,0.99,0.853,1/1
4249,1.912972e+07,242.1179,26.45,835,C15H15NO2,1.05,0.561,1/1
22029,8.107271e+06,484.1236,11.84,836,C24H26BrN3O3,1.01,0.899,1/1
22034,4.318005e+07,484.1236,11.84,836,C24H26BrN3O3,1.01,0.899,1/1


In [56]:
merged_df.columns[0]

'D:/QE/SA1.mzML'

In [57]:
non_duplicates = merged_df[~merged_df.duplicated(subset='Compound Name', keep=False)]
non_duplicates

,D:/QE/SA1.mzML,mz,RT,Compound Name,formula,Fold change,p-value,Compound concentration ratio
19210,1.081031e+07,455.1791,17.20,6,C20H22N8O5,0.93,0.098,1/1
11230,2.992829e+08,359.2447,0.58,8,C20H30N4O2,0.99,0.847,1/1
11316,4.357177e+06,360.9916,28.92,11,C14H11Cl3N2O3,0.95,0.463,1/1
2864,8.615798e+08,205.0975,3.34,13,C11H12N2O2,0.91,0.113,1/1
9803,3.274455e+08,338.0867,3.32,14,C15H20BrN3O,0.94,0.406,1/1
...,...,...,...,...,...,...,...,...
4025,6.936472e+08,237.1025,21.92,798,C15H12N2O,0.95,0.386,1/1
22344,4.100385e+07,487.1904,19.48,815,C24H25F3N6S,1.09,0.262,1/1
11892,3.474422e+06,369.0734,24.29,817,C19H16N2O2S2,0.95,0.424,1/1
4247,1.231051e+07,242.1175,25.26,834,C15H15NO2,0.99,0.853,1/1


In [58]:
duplicates = merged_df[merged_df.duplicated(subset='Compound Name', keep=False)]
duplicates

,D:/QE/SA1.mzML,mz,RT,Compound Name,formula,Fold change,p-value,Compound concentration ratio
12096,2.571572e+08,371.1617,18.85,1,C21H26N2S2,0.47,0.000,1/2
12103,6.198488e+09,371.1617,18.85,1,C21H26N2S2,0.47,0.000,1/2
12107,9.628062e+08,371.1617,18.85,1,C21H26N2S2,0.47,0.000,1/2
18031,2.227147e+07,442.2233,17.66,2,C25H31NO6,21.23,0.000,16/1
18034,3.657585e+08,442.2233,17.66,2,C25H31NO6,21.23,0.000,16/1
...,...,...,...,...,...,...,...,...
7935,9.168343e+08,310.1254,7.73,833,C19H19NOS,1.06,0.204,1/1
7939,1.025823e+08,310.1254,7.73,833,C19H19NOS,1.06,0.204,1/1
22029,8.107271e+06,484.1236,11.84,836,C24H26BrN3O3,1.01,0.899,1/1
22034,4.318005e+07,484.1236,11.84,836,C24H26BrN3O3,1.01,0.899,1/1


In [59]:
max_duplicates = duplicates.sort_values(by=['Compound Name', merged_df.columns[0]], ascending=[True, False]).drop_duplicates(subset='Compound Name', keep='first')
max_duplicates

,D:/QE/SA1.mzML,mz,RT,Compound Name,formula,Fold change,p-value,Compound concentration ratio
12103,6.198488e+09,371.1617,18.85,1,C21H26N2S2,0.47,0.000,1/2
18034,3.657585e+08,442.2233,17.66,2,C25H31NO6,21.23,0.000,16/1
23723,1.821726e+09,502.2395,22.78,4,C27H31N7OS,1.06,0.290,1/1
10093,1.365703e+09,342.1706,3.37,5,C20H23NO4,1.07,0.399,1/1
15008,1.305980e+07,408.1344,29.02,7,C21H24ClO6,0.84,0.157,1/1
...,...,...,...,...,...,...,...,...
17028,1.849716e+08,431.0498,13.39,830,C18H15ClN6OS2,0.95,0.440,1/1
12689,6.218351e+08,378.1360,13.68,831,C15H19N7O3S,0.53,0.312,1/2
13672,3.253145e+08,392.2339,7.05,832,C24H29N3O2,1.00,0.931,1/1
7935,9.168343e+08,310.1254,7.73,833,C19H19NOS,1.06,0.204,1/1


In [60]:
result = pd.concat([max_duplicates, non_duplicates])
result = result.drop_duplicates()
result_sorted= result.sort_values(by='Compound Name')
result_sorted

,D:/QE/SA1.mzML,mz,RT,Compound Name,formula,Fold change,p-value,Compound concentration ratio
12103,6.198488e+09,371.1617,18.85,1,C21H26N2S2,0.47,0.000,1/2
18034,3.657585e+08,442.2233,17.66,2,C25H31NO6,21.23,0.000,16/1
23723,1.821726e+09,502.2395,22.78,4,C27H31N7OS,1.06,0.290,1/1
10093,1.365703e+09,342.1706,3.37,5,C20H23NO4,1.07,0.399,1/1
19210,1.081031e+07,455.1791,17.20,6,C20H22N8O5,0.93,0.098,1/1
...,...,...,...,...,...,...,...,...
13672,3.253145e+08,392.2339,7.05,832,C24H29N3O2,1.00,0.931,1/1
7935,9.168343e+08,310.1254,7.73,833,C19H19NOS,1.06,0.204,1/1
4247,1.231051e+07,242.1175,25.26,834,C15H15NO2,0.99,0.853,1/1
4249,1.912972e+07,242.1179,26.45,835,C15H15NO2,1.05,0.561,1/1


In [15]:
# result.to_csv(r'D:\ms-peakonly-main\peakonly_results\peakonly_TOF\solved_TOFSB1.csv')

### merge TripleTOF 6600 of peakonly results

In [115]:
import os
import pandas as pd
solved_paths = r'D:\ms-peakonly-main\peakonly_results\peakonly_TOF'
# solved_paths = r'D:\ms-peakonly-main\peakonly_results\peakonly_QE'
files = os.listdir(solved_paths)
file = files[:8]
file

['solved_TOFSA1.csv',
 'solved_TOFSA2.csv',
 'solved_TOFSA3.csv',
 'solved_TOFSA4.csv',
 'solved_TOFSB1.csv',
 'solved_TOFSB2.csv',
 'solved_TOFSB3.csv',
 'solved_TOFSB4.csv']

In [116]:
first_file_path = os.path.join(solved_paths, files[0])
first_df = pd.read_csv(first_file_path)
col_name = 'Compound Name'
first_df.insert(1, col_name, first_df.pop(col_name))
first_df = first_df.iloc[:,:2]
first_df

,D:/TOF/20170326-960MIX_SampleA_1.mzML,Compound Name
0,475049.0,3
1,14722558.0,4
2,12847293.0,6
3,16974791.0,7
4,236502.0,8
...,...,...
803,1375940.0,960
804,455704.0,961
805,24756.0,963
806,480794.0,964


In [117]:
TOF_peakonly_inner = first_df
for i in range(7):
    temp_path = os.path.join(solved_paths, files[i+1])
    temp_df = pd.read_csv(temp_path).iloc[:, :2]
    TOF_peakonly_inner = pd.merge(TOF_peakonly_inner, temp_df, on=TOF_peakonly_inner.columns[1], how='inner')
TOF_peakonly_inner

,D:/TOF/20170326-960MIX_SampleA_1.mzML,Compound Name,D:/TOF/20170326-960MIX_SampleA_2.mzML,D:/TOF/SampleA_3-20170326-960MIX_SampleA_4.mzML,D:/TOF/SampleA_4-20170326-960MIX_SampleA_5.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_1.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_2.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_3.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_4.mzML
0,475049.0,3,399499.0,349886.0,475236.0,265657.0,223671.0,219552.0,308168.0
1,12847293.0,6,14692747.0,15509444.0,10061764.0,3575207.0,3549098.0,4001856.0,3391469.0
2,236502.0,8,298437.0,194044.0,198934.0,130205.0,120896.0,143272.0,148042.0
3,2392921.0,10,2633991.0,2010662.0,2207183.0,10932429.0,7885904.0,7947866.0,10172059.0
4,778702.0,24,727036.0,987609.0,845010.0,199995.0,182667.0,264480.0,265227.0
...,...,...,...,...,...,...,...,...,...
445,623504.0,943,2401342.0,3143624.0,3558719.0,2861334.0,3702959.0,2952937.0,3166649.0
446,3024317.0,948,2247063.0,2597790.0,2409201.0,3135509.0,2170083.0,1918322.0,3209504.0
447,3169412.0,951,2710509.0,2823546.0,2502218.0,3009576.0,1440948.0,2604951.0,1984568.0
448,1097629.0,953,1151651.0,711704.0,1298977.0,1335750.0,1394447.0,1404814.0,1743350.0


In [84]:
first_df

,D:/QE/SA1.mzML,Compound Name
0,6.198488e+09,1
1,3.657585e+08,2
2,1.821726e+09,4
3,1.365703e+09,5
4,1.305980e+07,7
...,...,...
810,6.936472e+08,798
811,4.100385e+07,815
812,3.474422e+06,817
813,1.231051e+07,834


In [87]:
QE_peakonly_inner = first_df
for i in range(9):
    temp_path = os.path.join(solved_paths, files[11+i])
    temp_df = pd.read_csv(temp_path)
    col_name = 'Compound Name'
    temp_df.insert(1, col_name, temp_df.pop(col_name))
    temp_df = temp_df.iloc[:,:2]
    QE_peakonly_inner = pd.merge(QE_peakonly_inner, temp_df, on=QE_peakonly_inner.columns[1], how='inner')
QE_peakonly_inner

,D:/QE/SA1.mzML,Compound Name,D:/QE/SA2.mzML,D:/QE/SA3.mzML,D:/QE/SA4.mzML,D:/QE/SA5.mzML,D:/QE/SB1.mzML,D:/QE/SB2.mzML,D:/QE/SB3.mzML,D:/QE/SB4.mzML,D:/QE/SB5.mzML
0,6.198488e+09,1,6.290617e+09,5.672294e+09,5.463803e+09,7.260214e+09,2.509768e+09,3.240872e+09,2.673348e+09,3.741343e+09,2.424612e+09
1,3.657585e+08,2,3.641640e+08,3.693638e+08,3.725333e+08,3.816289e+08,6.464050e+09,7.538872e+09,8.526330e+09,6.233312e+09,6.685903e+09
2,1.821726e+09,4,1.905309e+09,1.624257e+09,1.644665e+09,2.459561e+09,1.650160e+09,1.961893e+09,1.747589e+09,2.016725e+09,1.610263e+09
3,1.365703e+09,5,1.438979e+09,1.235628e+09,1.308550e+09,1.186266e+09,1.210546e+09,1.235240e+09,1.451471e+09,1.274914e+09,1.365750e+09
4,1.301760e+08,9,1.670175e+08,1.237498e+08,1.261011e+08,1.293953e+08,1.232948e+08,9.766323e+07,1.153561e+08,1.205442e+08,1.096642e+08
...,...,...,...,...,...,...,...,...,...,...,...
770,4.162060e+08,797,4.178905e+08,4.076364e+08,4.110515e+08,4.883098e+08,4.046506e+08,4.925159e+08,4.197165e+08,4.351342e+08,4.287377e+08
771,6.936472e+08,798,5.706304e+08,6.739700e+08,8.591281e+07,7.696168e+08,5.472128e+08,1.741635e+07,6.541056e+08,6.175587e+08,6.684427e+08
772,4.100385e+07,815,3.847857e+07,4.784769e+07,4.043739e+07,5.504471e+07,4.143656e+07,6.517926e+07,4.433957e+07,4.955423e+07,4.249550e+07
773,1.231051e+07,834,1.221731e+07,1.169823e+07,1.367304e+07,1.181374e+07,1.221436e+07,1.206983e+07,1.291697e+07,1.258379e+07,1.266561e+07


In [118]:
# 假设第二列的名字是 'col2'
col_name = 'Compound Name'
TOF_peakonly_inner.insert(0, col_name, TOF_peakonly_inner.pop(col_name))
TOF_peakonly_inner

,Compound Name,D:/TOF/20170326-960MIX_SampleA_1.mzML,D:/TOF/20170326-960MIX_SampleA_2.mzML,D:/TOF/SampleA_3-20170326-960MIX_SampleA_4.mzML,D:/TOF/SampleA_4-20170326-960MIX_SampleA_5.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_1.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_2.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_3.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_4.mzML
0,3,475049.0,399499.0,349886.0,475236.0,265657.0,223671.0,219552.0,308168.0
1,6,12847293.0,14692747.0,15509444.0,10061764.0,3575207.0,3549098.0,4001856.0,3391469.0
2,8,236502.0,298437.0,194044.0,198934.0,130205.0,120896.0,143272.0,148042.0
3,10,2392921.0,2633991.0,2010662.0,2207183.0,10932429.0,7885904.0,7947866.0,10172059.0
4,24,778702.0,727036.0,987609.0,845010.0,199995.0,182667.0,264480.0,265227.0
...,...,...,...,...,...,...,...,...,...
445,943,623504.0,2401342.0,3143624.0,3558719.0,2861334.0,3702959.0,2952937.0,3166649.0
446,948,3024317.0,2247063.0,2597790.0,2409201.0,3135509.0,2170083.0,1918322.0,3209504.0
447,951,3169412.0,2710509.0,2823546.0,2502218.0,3009576.0,1440948.0,2604951.0,1984568.0
448,953,1097629.0,1151651.0,711704.0,1298977.0,1335750.0,1394447.0,1404814.0,1743350.0


In [88]:
# 假设第二列的名字是 'col2'
col_name = 'Compound Name'
QE_peakonly_inner.insert(0, col_name, QE_peakonly_inner.pop(col_name))
QE_peakonly_inner

,Compound Name,D:/QE/SA1.mzML,D:/QE/SA2.mzML,D:/QE/SA3.mzML,D:/QE/SA4.mzML,D:/QE/SA5.mzML,D:/QE/SB1.mzML,D:/QE/SB2.mzML,D:/QE/SB3.mzML,D:/QE/SB4.mzML,D:/QE/SB5.mzML
0,1,6.198488e+09,6.290617e+09,5.672294e+09,5.463803e+09,7.260214e+09,2.509768e+09,3.240872e+09,2.673348e+09,3.741343e+09,2.424612e+09
1,2,3.657585e+08,3.641640e+08,3.693638e+08,3.725333e+08,3.816289e+08,6.464050e+09,7.538872e+09,8.526330e+09,6.233312e+09,6.685903e+09
2,4,1.821726e+09,1.905309e+09,1.624257e+09,1.644665e+09,2.459561e+09,1.650160e+09,1.961893e+09,1.747589e+09,2.016725e+09,1.610263e+09
3,5,1.365703e+09,1.438979e+09,1.235628e+09,1.308550e+09,1.186266e+09,1.210546e+09,1.235240e+09,1.451471e+09,1.274914e+09,1.365750e+09
4,9,1.301760e+08,1.670175e+08,1.237498e+08,1.261011e+08,1.293953e+08,1.232948e+08,9.766323e+07,1.153561e+08,1.205442e+08,1.096642e+08
...,...,...,...,...,...,...,...,...,...,...,...
770,797,4.162060e+08,4.178905e+08,4.076364e+08,4.110515e+08,4.883098e+08,4.046506e+08,4.925159e+08,4.197165e+08,4.351342e+08,4.287377e+08
771,798,6.936472e+08,5.706304e+08,6.739700e+08,8.591281e+07,7.696168e+08,5.472128e+08,1.741635e+07,6.541056e+08,6.175587e+08,6.684427e+08
772,815,4.100385e+07,3.847857e+07,4.784769e+07,4.043739e+07,5.504471e+07,4.143656e+07,6.517926e+07,4.433957e+07,4.955423e+07,4.249550e+07
773,834,1.231051e+07,1.221731e+07,1.169823e+07,1.367304e+07,1.181374e+07,1.221436e+07,1.206983e+07,1.291697e+07,1.258379e+07,1.266561e+07


In [89]:
df_TOF = pd.read_csv(r"D:\QuanFormer-main\data\FeatureTOF.csv")
df_QE = pd.read_csv(r"D:\QuanFormer-main\data\FeatureQE.csv")
df_TOF

,Compound Name,Compound formula,mz,RT,Fold change,Compound concentration ratio,p-value
0,1,C19H24N2S,313.1734,12.32,4.44,4/1,0.000
1,2,C24H30F2O6,453.2079,15.80,0.05,1/16,0.000
2,3,C26H32F2O7,495.2184,26.07,0.54,1/2,0.001
3,4,C21H23ClFNO2,376.1464,12.96,1.35,1/1,0.014
4,5,C26H28Cl2N4O4,531.1562,12.46,1.17,1/1,0.284
...,...,...,...,...,...,...,...
965,966,C17H12N2O2,299.0833,10.00,1.36,1/1,0.064
966,967,C17H12N2O2,299.0792,23.27,1.37,1/1,0.000
967,968,C19H26O3,325.1770,6.07,1.27,1/1,0.044
968,969,C19H20N4O2,359.1428,17.22,0.97,1/1,0.692


In [119]:
merged_TOF_results = pd.merge(TOF_peakonly_inner, df_TOF, on = 'Compound Name',how='inner')
merged_TOF_results

,Compound Name,D:/TOF/20170326-960MIX_SampleA_1.mzML,D:/TOF/20170326-960MIX_SampleA_2.mzML,D:/TOF/SampleA_3-20170326-960MIX_SampleA_4.mzML,D:/TOF/SampleA_4-20170326-960MIX_SampleA_5.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_1.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_2.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_3.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_4.mzML,Compound formula,mz,RT,Fold change,Compound concentration ratio,p-value
0,3,475049.0,399499.0,349886.0,475236.0,265657.0,223671.0,219552.0,308168.0,C26H32F2O7,495.2184,26.07,0.54,1/2,0.001
1,6,12847293.0,14692747.0,15509444.0,10061764.0,3575207.0,3549098.0,4001856.0,3391469.0,C18H34N2O6S,407.2227,3.30,0.28,1/4,0.000
2,8,236502.0,298437.0,194044.0,198934.0,130205.0,120896.0,143272.0,148042.0,C22H32O3,345.2421,25.02,0.54,1/2,0.000
3,10,2392921.0,2633991.0,2010662.0,2207183.0,10932429.0,7885904.0,7947866.0,10172059.0,C18H14Cl4N2O,416.9905,24.58,3.91,4/1,0.000
4,24,778702.0,727036.0,987609.0,845010.0,199995.0,182667.0,264480.0,265227.0,C18H14N4O5S,399.0755,11.84,0.27,1/4,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,943,623504.0,2401342.0,3143624.0,3558719.0,2861334.0,3702959.0,2952937.0,3166649.0,C26H27N5O2,442.2304,6.71,1.12,1/1,0.010
446,948,3024317.0,2247063.0,2597790.0,2409201.0,3135509.0,2170083.0,1918322.0,3209504.0,C23H25N3O3S,424.1735,26.33,0.95,1/1,0.233
447,951,3169412.0,2710509.0,2823546.0,2502218.0,3009576.0,1440948.0,2604951.0,1984568.0,C18H18N4,291.1604,2.24,1.16,1/1,0.379
448,953,1097629.0,1151651.0,711704.0,1298977.0,1335750.0,1394447.0,1404814.0,1743350.0,C15H19N7O3S,378.1369,13.77,1.20,1/1,0.010


In [90]:
merged_QE_results = pd.merge(QE_peakonly_inner, df_QE, on = 'Compound Name',how='inner')
merged_QE_results

,Compound Name,D:/QE/SA1.mzML,D:/QE/SA2.mzML,D:/QE/SA3.mzML,D:/QE/SA4.mzML,D:/QE/SA5.mzML,D:/QE/SB1.mzML,D:/QE/SB2.mzML,D:/QE/SB3.mzML,D:/QE/SB4.mzML,D:/QE/SB5.mzML,mz,RT,formula,Fold change,p-value,Compound concentration ratio
0,1,6.198488e+09,6.290617e+09,5.672294e+09,5.463803e+09,7.260214e+09,2.509768e+09,3.240872e+09,2.673348e+09,3.741343e+09,2.424612e+09,371.1617,18.85,C21H26N2S2,0.47,0.000,1/2
1,2,3.657585e+08,3.641640e+08,3.693638e+08,3.725333e+08,3.816289e+08,6.464050e+09,7.538872e+09,8.526330e+09,6.233312e+09,6.685903e+09,442.2233,17.66,C25H31NO6,21.23,0.000,16/1
2,4,1.821726e+09,1.905309e+09,1.624257e+09,1.644665e+09,2.459561e+09,1.650160e+09,1.961893e+09,1.747589e+09,2.016725e+09,1.610263e+09,502.2395,22.78,C27H31N7OS,1.06,0.290,1/1
3,5,1.365703e+09,1.438979e+09,1.235628e+09,1.308550e+09,1.186266e+09,1.210546e+09,1.235240e+09,1.451471e+09,1.274914e+09,1.365750e+09,342.1706,3.37,C20H23NO4,1.07,0.399,1/1
4,9,1.301760e+08,1.670175e+08,1.237498e+08,1.261011e+08,1.293953e+08,1.232948e+08,9.766323e+07,1.153561e+08,1.205442e+08,1.096642e+08,527.3126,26.96,C31H38N6O2,0.85,0.151,1/1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,797,4.162060e+08,4.178905e+08,4.076364e+08,4.110515e+08,4.883098e+08,4.046506e+08,4.925159e+08,4.197165e+08,4.351342e+08,4.287377e+08,227.1139,10.77,C9H14N4O3,1.00,0.968,1/1
771,798,6.936472e+08,5.706304e+08,6.739700e+08,8.591281e+07,7.696168e+08,5.472128e+08,1.741635e+07,6.541056e+08,6.175587e+08,6.684427e+08,237.1025,21.92,C15H12N2O,0.95,0.386,1/1
772,815,4.100385e+07,3.847857e+07,4.784769e+07,4.043739e+07,5.504471e+07,4.143656e+07,6.517926e+07,4.433957e+07,4.955423e+07,4.249550e+07,487.1904,19.48,C24H25F3N6S,1.09,0.262,1/1
773,834,1.231051e+07,1.221731e+07,1.169823e+07,1.367304e+07,1.181374e+07,1.221436e+07,1.206983e+07,1.291697e+07,1.258379e+07,1.266561e+07,242.1175,25.26,C15H15NO2,0.99,0.853,1/1


In [120]:
merged_TOF_results.loc[:,'AVG-A'] = merged_TOF_results.iloc[:,1:5].mean(axis=1)
merged_TOF_results.loc[:,'AVG-B'] = merged_TOF_results.iloc[:,5:9].mean(axis=1)
merged_TOF_results.loc[:,'CV-A'] = merged_TOF_results.iloc[:,1:5].std(axis=1)/merged_TOF_results.loc[:,'AVG-A']*100
merged_TOF_results.loc[:,'CV-B'] = merged_TOF_results.iloc[:,5:9].std(axis=1)/merged_TOF_results.loc[:,'AVG-B']*100
merged_TOF_results.loc[:,'fc'] = merged_TOF_results.loc[:,'AVG-B'] / merged_TOF_results.loc[:,'AVG-A']
merged_TOF_results.loc[:,'ratio'] = merged_TOF_results.loc[:,'fc'] / merged_TOF_results.loc[:,'Fold change']

In [91]:
merged_QE_results.loc[:,'AVG-A'] = merged_QE_results.iloc[:,1:6].mean(axis=1)
merged_QE_results.loc[:,'AVG-B'] = merged_QE_results.iloc[:,6:11].mean(axis=1)
merged_QE_results.loc[:,'CV-A'] = merged_QE_results.iloc[:,1:6].std(axis=1)/merged_QE_results.loc[:,'AVG-A']*100
merged_QE_results.loc[:,'CV-B'] = merged_QE_results.iloc[:,6:11].std(axis=1)/merged_QE_results.loc[:,'AVG-B']*100
merged_QE_results.loc[:,'fc'] = merged_QE_results.loc[:,'AVG-B'] / merged_QE_results.loc[:,'AVG-A']
merged_QE_results.loc[:,'ratio'] = merged_QE_results.loc[:,'fc'] / merged_QE_results.loc[:,'Fold change']

In [121]:
merged_TOF_results

,Compound Name,D:/TOF/20170326-960MIX_SampleA_1.mzML,D:/TOF/20170326-960MIX_SampleA_2.mzML,D:/TOF/SampleA_3-20170326-960MIX_SampleA_4.mzML,D:/TOF/SampleA_4-20170326-960MIX_SampleA_5.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_1.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_2.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_3.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_4.mzML,Compound formula,...,RT,Fold change,Compound concentration ratio,p-value,AVG-A,AVG-B,CV-A,CV-B,fc,ratio
0,3,475049.0,399499.0,349886.0,475236.0,265657.0,223671.0,219552.0,308168.0,C26H32F2O7,...,26.07,0.54,1/2,0.001,424917.50,254262.00,14.456930,16.336818,0.598380,1.108111
1,6,12847293.0,14692747.0,15509444.0,10061764.0,3575207.0,3549098.0,4001856.0,3391469.0,C18H34N2O6S,...,3.30,0.28,1/4,0.000,13277812.00,3629407.50,18.195439,7.197534,0.273344,0.976228
2,8,236502.0,298437.0,194044.0,198934.0,130205.0,120896.0,143272.0,148042.0,C22H32O3,...,25.02,0.54,1/2,0.000,231979.25,135603.75,20.775394,9.121430,0.584551,1.082502
3,10,2392921.0,2633991.0,2010662.0,2207183.0,10932429.0,7885904.0,7947866.0,10172059.0,C18H14Cl4N2O,...,24.58,3.91,4/1,0.000,2311189.25,9234564.50,11.502365,16.818055,3.995590,1.021890
4,24,778702.0,727036.0,987609.0,845010.0,199995.0,182667.0,264480.0,265227.0,C18H14N4O5S,...,11.84,0.27,1/4,0.000,834589.25,228092.25,13.523277,18.867253,0.273299,1.012218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,943,623504.0,2401342.0,3143624.0,3558719.0,2861334.0,3702959.0,2952937.0,3166649.0,C26H27N5O2,...,6.71,1.12,1/1,0.010,2431797.25,3170969.75,53.339662,11.889899,1.303961,1.164251
446,948,3024317.0,2247063.0,2597790.0,2409201.0,3135509.0,2170083.0,1918322.0,3209504.0,C23H25N3O3S,...,26.33,0.95,1/1,0.233,2569592.75,2608354.50,13.049571,25.310079,1.015085,1.068510
447,951,3169412.0,2710509.0,2823546.0,2502218.0,3009576.0,1440948.0,2604951.0,1984568.0,C18H18N4,...,2.24,1.16,1/1,0.379,2801421.25,2260010.75,9.962883,30.522965,0.806737,0.695463
448,953,1097629.0,1151651.0,711704.0,1298977.0,1335750.0,1394447.0,1404814.0,1743350.0,C15H19N7O3S,...,13.77,1.20,1/1,0.010,1064990.25,1469590.25,23.514194,12.590081,1.379910,1.149925


In [92]:
merged_QE_results

,Compound Name,D:/QE/SA1.mzML,D:/QE/SA2.mzML,D:/QE/SA3.mzML,D:/QE/SA4.mzML,D:/QE/SA5.mzML,D:/QE/SB1.mzML,D:/QE/SB2.mzML,D:/QE/SB3.mzML,D:/QE/SB4.mzML,...,formula,Fold change,p-value,Compound concentration ratio,AVG-A,AVG-B,CV-A,CV-B,fc,ratio
0,1,6.198488e+09,6.290617e+09,5.672294e+09,5.463803e+09,7.260214e+09,2.509768e+09,3.240872e+09,2.673348e+09,3.741343e+09,...,C21H26N2S2,0.47,0.000,1/2,6.177083e+09,2.917989e+09,11.303557,19.173548,0.472389,1.005084
1,2,3.657585e+08,3.641640e+08,3.693638e+08,3.725333e+08,3.816289e+08,6.464050e+09,7.538872e+09,8.526330e+09,6.233312e+09,...,C25H31NO6,21.23,0.000,16/1,3.706897e+08,7.089693e+09,1.867662,13.294557,19.125683,0.900880
2,4,1.821726e+09,1.905309e+09,1.624257e+09,1.644665e+09,2.459561e+09,1.650160e+09,1.961893e+09,1.747589e+09,2.016725e+09,...,C27H31N7OS,1.06,0.290,1/1,1.891103e+09,1.797326e+09,17.934104,10.196469,0.950411,0.896614
3,5,1.365703e+09,1.438979e+09,1.235628e+09,1.308550e+09,1.186266e+09,1.210546e+09,1.235240e+09,1.451471e+09,1.274914e+09,...,C20H23NO4,1.07,0.399,1/1,1.307025e+09,1.307584e+09,7.702225,7.628955,1.000428,0.934979
4,9,1.301760e+08,1.670175e+08,1.237498e+08,1.261011e+08,1.293953e+08,1.232948e+08,9.766323e+07,1.153561e+08,1.205442e+08,...,C31H38N6O2,0.85,0.151,1/1,1.352879e+08,1.133045e+08,13.249132,8.982472,0.837506,0.985302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,797,4.162060e+08,4.178905e+08,4.076364e+08,4.110515e+08,4.883098e+08,4.046506e+08,4.925159e+08,4.197165e+08,4.351342e+08,...,C9H14N4O3,1.00,0.968,1/1,4.282188e+08,4.361510e+08,7.902224,7.686269,1.018524,1.018524
771,798,6.936472e+08,5.706304e+08,6.739700e+08,8.591281e+07,7.696168e+08,5.472128e+08,1.741635e+07,6.541056e+08,6.175587e+08,...,C15H12N2O,0.95,0.386,1/1,5.587554e+08,5.009472e+08,48.984443,54.764553,0.896541,0.943728
772,815,4.100385e+07,3.847857e+07,4.784769e+07,4.043739e+07,5.504471e+07,4.143656e+07,6.517926e+07,4.433957e+07,4.955423e+07,...,C24H25F3N6S,1.09,0.262,1/1,4.456244e+07,4.860102e+07,15.358336,20.120924,1.090627,1.000576
773,834,1.231051e+07,1.221731e+07,1.169823e+07,1.367304e+07,1.181374e+07,1.221436e+07,1.206983e+07,1.291697e+07,1.258379e+07,...,C15H15NO2,0.99,0.853,1/1,1.234257e+07,1.249011e+07,6.381728,2.757413,1.011954,1.022176


In [125]:
from scipy import stats

# 创建新的列来存储 t 统计量和 p 值
merged_TOF_results['t_statistic'] = 0
merged_TOF_results['p_value'] = 0

# 对每一行进行 t 检验
for index, row in merged_TOF_results.iterrows():
    cols_1_to_5 = row[1:5].values.astype(float)
    cols_5_to_9 = row[5:9].values.astype(float)
    
    # 进行 t 检验
    t_statistic, p_value = stats.ttest_ind(cols_1_to_5, cols_5_to_9)
    merged_TOF_results.at[index, 't_statistic'] = t_statistic
    merged_TOF_results.at[index, 'p_value'] = p_value

In [126]:
from scipy import stats

# 创建新的列来存储 t 统计量和 p 值
merged_QE_results['t_statistic'] = 0
merged_QE_results['p_value'] = 0

# 对每一行进行 t 检验
for index, row in merged_QE_results.iterrows():
    cols_1_to_6 = row[0:6].values.astype(float)
    cols_6_to_11 = row[6:11].values.astype(float)
    
    # 进行 t 检验
    t_statistic, p_value = stats.ttest_ind(cols_1_to_6, cols_6_to_11)
    
    # 存储结果
    merged_QE_results.at[index, 't_statistic'] = t_statistic
    merged_QE_results.at[index, 'p_value'] = p_value

In [127]:
merged_TOF_results

,Compound Name,D:/TOF/20170326-960MIX_SampleA_1.mzML,D:/TOF/20170326-960MIX_SampleA_2.mzML,D:/TOF/SampleA_3-20170326-960MIX_SampleA_4.mzML,D:/TOF/SampleA_4-20170326-960MIX_SampleA_5.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_1.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_2.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_3.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_4.mzML,Compound formula,...,Compound concentration ratio,p-value,AVG-A,AVG-B,CV-A,CV-B,fc,ratio,t_statistic,p_value
0,3,475049.0,399499.0,349886.0,475236.0,265657.0,223671.0,219552.0,308168.0,C26H32F2O7,...,1/2,0.001,424917.50,254262.00,14.456930,16.336818,0.598380,1.108111,4.602621,0.003682
1,6,12847293.0,14692747.0,15509444.0,10061764.0,3575207.0,3549098.0,4001856.0,3391469.0,C18H34N2O6S,...,1/4,0.000,13277812.00,3629407.50,18.195439,7.197534,0.273344,0.976228,7.940950,0.000212
2,8,236502.0,298437.0,194044.0,198934.0,130205.0,120896.0,143272.0,148042.0,C22H32O3,...,1/2,0.000,231979.25,135603.75,20.775394,9.121430,0.584551,1.082502,3.873884,0.008229
3,10,2392921.0,2633991.0,2010662.0,2207183.0,10932429.0,7885904.0,7947866.0,10172059.0,C18H14Cl4N2O,...,4/1,0.000,2311189.25,9234564.50,11.502365,16.818055,3.995590,1.021890,-8.787893,0.000120
4,24,778702.0,727036.0,987609.0,845010.0,199995.0,182667.0,264480.0,265227.0,C18H14N4O5S,...,1/4,0.000,834589.25,228092.25,13.523277,18.867253,0.273299,1.012218,10.042170,0.000057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,943,623504.0,2401342.0,3143624.0,3558719.0,2861334.0,3702959.0,2952937.0,3166649.0,C26H27N5O2,...,1/1,0.010,2431797.25,3170969.75,53.339662,11.889899,1.303961,1.164251,-1.094425,0.315746
446,948,3024317.0,2247063.0,2597790.0,2409201.0,3135509.0,2170083.0,1918322.0,3209504.0,C23H25N3O3S,...,1/1,0.233,2569592.75,2608354.50,13.049571,25.310079,1.015085,1.068510,-0.104697,0.920028
447,951,3169412.0,2710509.0,2823546.0,2502218.0,3009576.0,1440948.0,2604951.0,1984568.0,C18H18N4,...,1/1,0.379,2801421.25,2260010.75,9.962883,30.522965,0.806737,0.695463,1.455120,0.195877
448,953,1097629.0,1151651.0,711704.0,1298977.0,1335750.0,1394447.0,1404814.0,1743350.0,C15H19N7O3S,...,1/1,0.010,1064990.25,1469590.25,23.514194,12.590081,1.379910,1.149925,-2.598915,0.040722


In [128]:
merged_QE_results

,Compound Name,D:/QE/SA1.mzML,D:/QE/SA2.mzML,D:/QE/SA3.mzML,D:/QE/SA4.mzML,D:/QE/SA5.mzML,D:/QE/SB1.mzML,D:/QE/SB2.mzML,D:/QE/SB3.mzML,D:/QE/SB4.mzML,...,Compound concentration ratio,AVG-A,AVG-B,CV-A,CV-B,fc,ratio,t_statistic,p_value,corrected_p_value
0,1,6.198488e+09,6.290617e+09,5.672294e+09,5.463803e+09,7.260214e+09,2.509768e+09,3.240872e+09,2.673348e+09,3.741343e+09,...,1/2,6.177083e+09,2.917989e+09,11.303557,19.173548,0.472389,1.005084,1.867153,9.472749e-02,0.816432
1,2,3.657585e+08,3.641640e+08,3.693638e+08,3.725333e+08,3.816289e+08,6.464050e+09,7.538872e+09,8.526330e+09,6.233312e+09,...,16/1,3.706897e+08,7.089693e+09,1.867662,13.294557,19.125683,0.900880,-17.540256,2.879454e-08,0.000003
2,4,1.821726e+09,1.905309e+09,1.624257e+09,1.644665e+09,2.459561e+09,1.650160e+09,1.961893e+09,1.747589e+09,2.016725e+09,...,1/1,1.891103e+09,1.797326e+09,17.934104,10.196469,0.950411,0.896614,-0.580174,5.760325e-01,0.842312
3,5,1.365703e+09,1.438979e+09,1.235628e+09,1.308550e+09,1.186266e+09,1.210546e+09,1.235240e+09,1.451471e+09,1.274914e+09,...,1/1,1.307025e+09,1.307584e+09,7.702225,7.628955,1.000428,0.934979,-0.882300,4.005656e-01,0.816432
4,9,1.301760e+08,1.670175e+08,1.237498e+08,1.261011e+08,1.293953e+08,1.232948e+08,9.766323e+07,1.153561e+08,1.205442e+08,...,1/1,1.352879e+08,1.133045e+08,13.249132,8.982472,0.837506,0.985302,-0.021483,9.833293e-01,0.996183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,797,4.162060e+08,4.178905e+08,4.076364e+08,4.110515e+08,4.883098e+08,4.046506e+08,4.925159e+08,4.197165e+08,4.351342e+08,...,1/1,4.282188e+08,4.361510e+08,7.902224,7.686269,1.018524,1.018524,-0.976486,3.543389e-01,0.816432
771,798,6.936472e+08,5.706304e+08,6.739700e+08,8.591281e+07,7.696168e+08,5.472128e+08,1.741635e+07,6.541056e+08,6.175587e+08,...,1/1,5.587554e+08,5.009472e+08,48.984443,54.764553,0.896541,0.943728,-0.188584,8.546025e-01,0.936804
772,815,4.100385e+07,3.847857e+07,4.784769e+07,4.043739e+07,5.504471e+07,4.143656e+07,6.517926e+07,4.433957e+07,4.955423e+07,...,1/1,4.456244e+07,4.860102e+07,15.358336,20.120924,1.090627,1.000576,-1.204335,2.591753e-01,0.816432
773,834,1.231051e+07,1.221731e+07,1.169823e+07,1.367304e+07,1.181374e+07,1.221436e+07,1.206983e+07,1.291697e+07,1.258379e+07,...,1/1,1.234257e+07,1.249011e+07,6.381728,2.757413,1.011954,1.022176,-0.958322,3.629346e-01,0.816432


In [129]:
from statsmodels.stats.multitest import multipletests

In [130]:
# 提取所有行的 p 值
p_values = merged_TOF_results['p_value'].values

# 进行 FDR 校正
fdr_level = 0.05
reject, corrected_p_values, _, _ = multipletests(p_values, alpha=fdr_level, method='fdr_bh')

# 将校正后的 p 值添加到 DataFrame 中
merged_TOF_results['corrected_p_value'] = corrected_p_values
merged_TOF_results

,Compound Name,D:/TOF/20170326-960MIX_SampleA_1.mzML,D:/TOF/20170326-960MIX_SampleA_2.mzML,D:/TOF/SampleA_3-20170326-960MIX_SampleA_4.mzML,D:/TOF/SampleA_4-20170326-960MIX_SampleA_5.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_1.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_2.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_3.mzML,F:/TOF_MZML/20170326-960MIX_SampleB_4.mzML,Compound formula,...,p-value,AVG-A,AVG-B,CV-A,CV-B,fc,ratio,t_statistic,p_value,corrected_p_value
0,3,475049.0,399499.0,349886.0,475236.0,265657.0,223671.0,219552.0,308168.0,C26H32F2O7,...,0.001,424917.50,254262.00,14.456930,16.336818,0.598380,1.108111,4.602621,0.003682,0.057132
1,6,12847293.0,14692747.0,15509444.0,10061764.0,3575207.0,3549098.0,4001856.0,3391469.0,C18H34N2O6S,...,0.000,13277812.00,3629407.50,18.195439,7.197534,0.273344,0.976228,7.940950,0.000212,0.010600
2,8,236502.0,298437.0,194044.0,198934.0,130205.0,120896.0,143272.0,148042.0,C22H32O3,...,0.000,231979.25,135603.75,20.775394,9.121430,0.584551,1.082502,3.873884,0.008229,0.082466
3,10,2392921.0,2633991.0,2010662.0,2207183.0,10932429.0,7885904.0,7947866.0,10172059.0,C18H14Cl4N2O,...,0.000,2311189.25,9234564.50,11.502365,16.818055,3.995590,1.021890,-8.787893,0.000120,0.006772
4,24,778702.0,727036.0,987609.0,845010.0,199995.0,182667.0,264480.0,265227.0,C18H14N4O5S,...,0.000,834589.25,228092.25,13.523277,18.867253,0.273299,1.012218,10.042170,0.000057,0.003635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,943,623504.0,2401342.0,3143624.0,3558719.0,2861334.0,3702959.0,2952937.0,3166649.0,C26H27N5O2,...,0.010,2431797.25,3170969.75,53.339662,11.889899,1.303961,1.164251,-1.094425,0.315746,0.541889
446,948,3024317.0,2247063.0,2597790.0,2409201.0,3135509.0,2170083.0,1918322.0,3209504.0,C23H25N3O3S,...,0.233,2569592.75,2608354.50,13.049571,25.310079,1.015085,1.068510,-0.104697,0.920028,0.959715
447,951,3169412.0,2710509.0,2823546.0,2502218.0,3009576.0,1440948.0,2604951.0,1984568.0,C18H18N4,...,0.379,2801421.25,2260010.75,9.962883,30.522965,0.806737,0.695463,1.455120,0.195877,0.440724
448,953,1097629.0,1151651.0,711704.0,1298977.0,1335750.0,1394447.0,1404814.0,1743350.0,C15H19N7O3S,...,0.010,1064990.25,1469590.25,23.514194,12.590081,1.379910,1.149925,-2.598915,0.040722,0.185943


In [108]:
# 提取所有行的 p 值
p_values = merged_QE_results['p_value'].values

# 进行 FDR 校正
fdr_level = 0.05
reject, corrected_p_values, _, _ = multipletests(p_values, alpha=fdr_level, method='fdr_bh')

# 将校正后的 p 值添加到 DataFrame 中
merged_QE_results['corrected_p_value'] = corrected_p_values
merged_QE_results

,Compound Name,D:/QE/SA1.mzML,D:/QE/SA2.mzML,D:/QE/SA3.mzML,D:/QE/SA4.mzML,D:/QE/SA5.mzML,D:/QE/SB1.mzML,D:/QE/SB2.mzML,D:/QE/SB3.mzML,D:/QE/SB4.mzML,...,Compound concentration ratio,AVG-A,AVG-B,CV-A,CV-B,fc,ratio,t_statistic,p_value,corrected_p_value
0,1,6.198488e+09,6.290617e+09,5.672294e+09,5.463803e+09,7.260214e+09,2.509768e+09,3.240872e+09,2.673348e+09,3.741343e+09,...,1/2,6.177083e+09,2.917989e+09,11.303557,19.173548,0.472389,1.005084,0,9.472749e-02,0.816432
1,2,3.657585e+08,3.641640e+08,3.693638e+08,3.725333e+08,3.816289e+08,6.464050e+09,7.538872e+09,8.526330e+09,6.233312e+09,...,16/1,3.706897e+08,7.089693e+09,1.867662,13.294557,19.125683,0.900880,0,2.879454e-08,0.000003
2,4,1.821726e+09,1.905309e+09,1.624257e+09,1.644665e+09,2.459561e+09,1.650160e+09,1.961893e+09,1.747589e+09,2.016725e+09,...,1/1,1.891103e+09,1.797326e+09,17.934104,10.196469,0.950411,0.896614,0,5.760325e-01,0.842312
3,5,1.365703e+09,1.438979e+09,1.235628e+09,1.308550e+09,1.186266e+09,1.210546e+09,1.235240e+09,1.451471e+09,1.274914e+09,...,1/1,1.307025e+09,1.307584e+09,7.702225,7.628955,1.000428,0.934979,0,4.005656e-01,0.816432
4,9,1.301760e+08,1.670175e+08,1.237498e+08,1.261011e+08,1.293953e+08,1.232948e+08,9.766323e+07,1.153561e+08,1.205442e+08,...,1/1,1.352879e+08,1.133045e+08,13.249132,8.982472,0.837506,0.985302,0,9.833293e-01,0.996183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,797,4.162060e+08,4.178905e+08,4.076364e+08,4.110515e+08,4.883098e+08,4.046506e+08,4.925159e+08,4.197165e+08,4.351342e+08,...,1/1,4.282188e+08,4.361510e+08,7.902224,7.686269,1.018524,1.018524,0,3.543389e-01,0.816432
771,798,6.936472e+08,5.706304e+08,6.739700e+08,8.591281e+07,7.696168e+08,5.472128e+08,1.741635e+07,6.541056e+08,6.175587e+08,...,1/1,5.587554e+08,5.009472e+08,48.984443,54.764553,0.896541,0.943728,0,8.546025e-01,0.936804
772,815,4.100385e+07,3.847857e+07,4.784769e+07,4.043739e+07,5.504471e+07,4.143656e+07,6.517926e+07,4.433957e+07,4.955423e+07,...,1/1,4.456244e+07,4.860102e+07,15.358336,20.120924,1.090627,1.000576,0,2.591753e-01,0.816432
773,834,1.231051e+07,1.221731e+07,1.169823e+07,1.367304e+07,1.181374e+07,1.221436e+07,1.206983e+07,1.291697e+07,1.258379e+07,...,1/1,1.234257e+07,1.249011e+07,6.381728,2.757413,1.011954,1.022176,0,3.629346e-01,0.816432


In [131]:
df_precise = merged_TOF_results[(merged_TOF_results['ratio'] > 0.8) & (merged_TOF_results['ratio'] < 1.2)]
df_precise.shape[0]

310

In [109]:
df_precise = merged_QE_results[(merged_QE_results['ratio'] > 0.8) & (merged_QE_results['ratio'] < 1.2)]
df_precise.shape[0]

675

In [132]:
round((df_precise.shape[0]/merged_TOF_results.shape[0] + merged_TOF_results.shape[0] / 970)/2, 3)

0.576

In [110]:
round((df_precise.shape[0]/merged_QE_results.shape[0] + merged_QE_results.shape[0] / 836)/2, 3)

0.899

In [133]:
merged_TOF_results.to_csv(r'D:\ms-peakonly-main\peakonly_results\peakonly_TOF\inner.csv',index=False)

In [134]:
merged_QE_results.to_csv(r'D:\ms-peakonly-main\peakonly_results\peakonly_QE\inner.csv',index=False)

In [135]:
import pandas as pd

use = ['Fold change', 'p-value', 'fc', 'corrected_p_value','Compound Name']
df = pd.read_csv(r'D:\ms-peakonly-main\peakonly_results\peakonly_TOF\inner.csv', usecols=use, encoding_errors='ignore')

df1 = df[((df['Fold change'] > 2) | (df['Fold change'] < 0.5)) & (df['p-value'] < 0.05)]
set1 = set(df1['Compound Name'])
df2 = df[((df['fc'] > 2) | (df['fc'] < 0.5)) & (df['corrected_p_value'] < 0.05)]
set2 = set(df2['Compound Name'])
print(len(set1.intersection(set2)))
print(len(set1.union(set2))-len(set1.intersection(set2)))

14
15


In [112]:
import pandas as pd

use = ['Fold change', 'p-value', 'fc', 'corrected_p_value','Compound Name']
df = pd.read_csv(r'D:\ms-peakonly-main\peakonly_results\peakonly_QE\inner.csv', usecols=use, encoding_errors='ignore')

df1 = df[((df['Fold change'] > 2) | (df['Fold change'] < 0.5)) & (df['p-value'] < 0.05)]
set1 = set(df1['Compound Name'])
df2 = df[((df['fc'] > 2) | (df['fc'] < 0.5)) & (df['corrected_p_value'] < 0.05)]
set2 = set(df2['Compound Name'])
print(len(set1.intersection(set2)))
print(len(set1.union(set2))-len(set1.intersection(set2)))

17
31
